In [1]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species

In [2]:
database = RMGDatabase()
database.load('/home/kehang/Code/ipso_training/RMG-database/input', kineticsFamilies='all', kineticsDepositories='all')

In [6]:
reactionDict = {}
libraryName = 'vinylCPD_H'
kineticLibrary = database.kinetics.libraries[libraryName]
for index, entry in kineticLibrary.entries.iteritems():
    reaction = entry.item
    reaction.kinetics = entry.data    
    # Let's make RMG try to generate this reaction from the families!
    rmgReactionList = []
    rxt_mol_mutation_num = 1
    pdt_mol_mutation_num = 1
    for reactant in reaction.reactants:
        rxt_mol_mutation_num *= len(reactant.molecule)
    for product in reaction.products:
        pdt_mol_mutation_num *= len(product.molecule)
    
    for mutation_i in range(rxt_mol_mutation_num):
        rxts_mol = [spc.molecule[mutation_i%(len(spc.molecule))] for spc in reaction.reactants]
        for mutation_j in range(pdt_mol_mutation_num):
            pdts_mol = [spc.molecule[mutation_j%(len(spc.molecule))] for spc in reaction.products]
            rmgReactionList.extend(database.kinetics.generateReactionsFromFamilies(
                    reactants=rxts_mol, products=pdts_mol))
    
    if len(rmgReactionList) == 1:
        rmgReaction = rmgReactionList[0]
        if rmgReaction.family in reactionDict:
            reactionDict[rmgReaction.family].append(reaction)
        else:
            reactionDict[rmgReaction.family] = [reaction]
    elif len(rmgReactionList) == 0:
        print reaction
        print 'reactants'
        for reactant in reaction.reactants:
            print reactant.molecule[0].toSMILES()
        print 'products'
        for product in reaction.products:
            print product.molecule[0].toSMILES()
        print "Sad :( There are no matches.  This is a magic reaction or has chemistry that should be made into a new reaction family"
    else:
        if rmgReactionList[0].family not in ['Intra_R_Add_Exocyclic','intra_H_migration']:
            print reaction
            for rxn in rmgReactionList:
                print rxn.family
            print "There are multiple RMG matches for this reaction. You have to manually create this training reaction"

vinylCPD + H <=> addC
R_Addition_MultipleBond
R_Addition_MultipleBond
There are multiple RMG matches for this reaction. You have to manually create this training reaction
addC <=> product16
Intra_R_Add_Endocyclic
Intra_R_Add_Endocyclic
Intra_R_Add_Endocyclic
Intra_R_Add_Endocyclic
There are multiple RMG matches for this reaction. You have to manually create this training reaction
addC <=> product6
Intra_RH_Add_Endocyclic
Intra_R_Add_Endocyclic
Intra_RH_Add_Exocyclic
There are multiple RMG matches for this reaction. You have to manually create this training reaction
product2 <=> BENZENE + CH3
R_Addition_MultipleBond
R_Addition_MultipleBond
There are multiple RMG matches for this reaction. You have to manually create this training reaction
product2 <=> TOLUENE + H
R_Addition_MultipleBond
R_Addition_MultipleBond
There are multiple RMG matches for this reaction. You have to manually create this training reaction
product10 <=> product11
Intra_R_Add_Endocyclic
Intra_R_Add_Endocyclic
Intra_R_

In [8]:
for familyName in reactionDict:
    print 'Adding training reactions for family: ' + familyName
    kineticFamily = database.kinetics.families[familyName]
    reactions = reactionDict[familyName]
    # save training reactions of one family
    kineticFamily.saveTrainingReaction(reactions)
    print str(reactions[0])
    break

Adding training reactions for family: R_Addition_MultipleBond
vinylCPD + H <=> addA
